In [4]:
from problem2 import ProblemSingleElementWise
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from mutation import MyMutation
from crossover import TraceCrossover
from encoder import Encoder
from sampling import MySampling
from callback import UpdatePopulationCallback
import numpy as np
from tools import Tools
from survival import MySurvival
from Declare4Py.ProcessModels.DeclareModel import DeclareModel
from Declare4Py.D4PyEventLog import D4PyEventLog
import warnings
import random
from Declare4Py.ProcessMiningTasks.ConformanceChecking.MPDeclareResultsBrowser import MPDeclareResultsBrowser
from Declare4Py.ProcessMiningTasks.ConformanceChecking.MPDeclareAnalyzer import MPDeclareAnalyzer
import pandas as pd
import logging
from terminator import MyTermination
from problem import Problem_single_ElementWise, MyProblem_Problem, MyProblem_Problem2, Problem_multi_ElementWise, Problem_single_ElementWise_noConstraints
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.termination.default import DefaultSingleObjectiveTermination, DefaultMultiObjectiveTermination
import testSetup
from pymoo.operators.mutation.pm import PolynomialMutation
from pymoo.operators.crossover.sbx import SBX
from mutation import IntegerPolynomialMutation
from testSetup import Setup
from pymoo.core.duplicate import ElementwiseDuplicateElimination

warnings.filterwarnings("ignore", ".*feasible.*")
logging.getLogger('matplotlib').setLevel(logging.WARNING)

In [5]:
from problem2 import ProblemMultiNoConstElementWise, ProblemMultiElementWise

# setup population size and number of events per trace
pop_size = 3000
trace_length = 50



# setup shared components and initial population
(
    encoder, declare, event_log, dataframe, activities_name
) = Setup.initialize_shared_components(path_to_declareModel="../declare_models/model1.decl", trace_length=trace_length)
(
    initial_population, initial_encoded_pop, lower_bounds, upper_bounds, mutation, crossover, sampling
) = Setup.setup_initial_population(activities_name=activities_name, encoder=encoder)


class MyDuplicateElimination(ElementwiseDuplicateElimination):

    def is_equal(self, a, b):
        return a.X[0] == b.X[0]

# setup termination
termination_multi = DefaultMultiObjectiveTermination(
    xtol=1e-8,
    cvtol=1e-6,
    ftol=0.0025,
    period=30,
    n_max_gen=1000,
    n_max_evals=100000
)
termination_single = DefaultSingleObjectiveTermination(
    xtol=1e-8,
    cvtol=1e-6,
    ftol=1e-6,
    period=20,
    n_max_gen=1000,
    n_max_evals=100000
)
termination = MyTermination(n_required=pop_size*0.5)


# setup problem
multi_problem = ProblemMultiElementWise(
    trace_length=trace_length,
    encoder=encoder,
    d4py=declare,
    initial_population=initial_encoded_pop,
    xl=lower_bounds,xu=upper_bounds,
    event_log=event_log,
    dataframe=dataframe
)
single_problem = ProblemSingleElementWise(
    trace_length=trace_length,
    encoder=encoder,
    d4py=declare,
    initial_population=initial_encoded_pop,
    xl=lower_bounds,xu=upper_bounds,
    event_log=event_log,
    dataframe=dataframe
)

# setup algorithm
multi_algorithm = NSGA2(
    problem=multi_problem,
    pop_size=pop_size,
    sampling=sampling,
    crossover=crossover,
    mutation=mutation,
    callback=UpdatePopulationCallback(),
    eliminate_duplicates=True,
)

single_algorithm = GA(
    problem=single_problem,
    pop_size=pop_size,
    sampling=sampling,
    crossover=crossover,
    mutation=mutation,
    callback=UpdatePopulationCallback(),
    eliminate_duplicates=False,
)

Tools.calculate_overall_diversity(encoder,"results/encoded_traces_2025-02-15/ID_3_run_1_multi_yes_constraints.csv", 50)

print(len(declare.activities))

Overall Diversity Score: 0.8186018977777778
13


In [6]:
result = minimize(multi_problem, multi_algorithm, termination=termination, seed=1, verbose=True)

Tools.save_simple_solution(result.pop, encoder)

data = result.algorithm.callback.get_data()

diversity_scores = data.get("diversity_history", None)
constraint_scores = data.get("constraint_history", None)
n_violations_scores = data.get("n_violations_history", None)
n_generations = data.get("generations", None)

print(f"Diversity score:{diversity_scores[-1]}")

n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |       10 |      1 |  0.000000E+00 |  0.000000E+00 |             - |             -
     2 |     3010 |      1 |  0.000000E+00 |  4.7086666667 |  0.000000E+00 |             f
     3 |     6010 |      1 |  0.000000E+00 |  4.2383333333 |  0.000000E+00 |             f
     4 |     9010 |      1 |  0.000000E+00 |  4.0020000000 |  0.000000E+00 |             f
     5 |    12010 |      1 |  0.000000E+00 |  3.7276666667 |  0.000000E+00 |             f
     6 |    15010 |      1 |  0.000000E+00 |  3.5926666667 |  0.000000E+00 |             f
     7 |    18010 |      1 |  0.000000E+00 |  3.5173333333 |  0.000000E+00 |             f
     8 |    21010 |      1 |  0.000000E+00 |  3.4420000000 |  0.000000E+00 |             f
     9 |    24010 |      1 |  0.000000E+00 |  3.3583333333 |  0.000000E+00 |             f
    10 |    27010 |      1 |  0.000000E+00 |  3.2680000000 |  0.000000E+00 |             f

KeyboardInterrupt: 